In [1]:
import sys
sys.path.append('../app/')

In [3]:
from pymongo import MongoClient
import pandas as pd
from feature_extractor import get_ohe_features
import joblib as jb
import datetime
import json

In [4]:
with open('../config.json') as json_file:
    config = json.load(json_file)

In [5]:
client = MongoClient(config['MONGO_URI'])
db = client.linkedin
collection = db.jobIds

In [3]:
df = pd.DataFrame(list(collection.find({ 'label' : {'$exists' : False}})))

In [ ]:
df.head(15)

In [4]:
model = jb.load('./models/lgbm_07_Jul_2020.jb')
desc_vec = jb.load('./models/desc_vec_07_Jul_2020.jb')
title_vec = jb.load('./models/title_vec_07_Jul_2020.jb')

In [5]:
usedFeatures = ['applicants', 'applicants_per_day', 'description_len']

features_list = [df[usedFeatures]]


In [6]:
categorical_fields = ['Seniority level', 'Employment type', 'Job function', 'Industries']

for field in categorical_fields:
    features_list.append(get_ohe_features(df, field))


In [7]:
title_bow_test = title_vec.transform(df['title'])
desc_bow_test = desc_vec.transform(df['description'])
features_list.append(title_bow_test)
features_list.append(desc_bow_test)

In [8]:
from scipy.sparse import hstack

X_test = hstack(features_list)

In [9]:
pred = model.predict_proba(X_test)[:, 1]

In [10]:
df['pred'] = pred

In [11]:
df.sort_values('pred', ascending=False).head(5)

,_id,jobId,loaded,company,city,posted,applicants,applicants_per_day,description,Seniority level,Employment type,Job function,Industries,title,pred,created,added,description_len,pred_date
446,5eff7c51a2ae983dbb5d6893,1906711546,True,Harnham,"London, England, United Kingdom",2020-07-04 23:28:12.838,28,28.000000,Data Scientist / Senior Data Scientist – Tempo...,Mid-Senior level,Full-time,Information Technology,Analyst,Data Scientist,0.893880,2020-07-04 23:28:12.838,2020-07-04 23:28:12.838,2075,2020-07-06 23:33:00.295
805,5f03a910f904fbf7622f3aeb,1932926640,True,Unicorn Hunt,"London, England, United Kingdom",2020-07-03 22:44:01.682,53,13.250000,What is Carra? Carra is a hyper-personalisatio...,Entry level,Full-time,Engineering,Information Technology,Data Scientist,0.893594,2020-07-06 22:44:01.682,2020-07-06 22:43:28.166,2568,2020-07-06 23:33:01.344
421,5eff7c51a2ae983dbb5d6878,1718334914,True,Apple,"London, England, United Kingdom",2020-02-15 23:28:12.784,10,0.070922,SummaryWe are looking for an individual with a...,Not Applicable,Full-time,Engineering,Information Technology,"Data Scientist - App Store, Apple Music, Apple...",0.884004,2020-07-04 23:28:12.784,2020-07-04 23:28:12.784,2332,2020-07-06 23:33:00.207
424,5eff7c51a2ae983dbb5d687c,1866261249,True,JUST EAT,"London, England, United Kingdom",2020-06-13 23:28:12.793,10,0.454545,Job Title: Data ScientistDepartment: Machine L...,Mid-Senior level,Full-time,Engineering,Information Technology,Data Scientist,0.866860,2020-07-04 23:28:12.793,2020-07-04 23:28:12.793,2761,2020-07-06 23:33:00.217
286,5efca9dbb4432a82d5b96eee,1877473764,True,Lendico Brazil,"São Paulo, Brazil",2020-06-06 23:28:12.448,10,0.344828,Este processo seletivo encontra-se em stand by...,Mid-Senior level,Full-time,Analyst,Strategy/Planning,Senior Business Intelligence Analyst I Talent ...,0.860732,2020-07-04 23:28:12.448,2020-07-04 23:28:12.448,2447,2020-07-06 23:32:59.786


In [ ]:
float(df[df['jobId'] == '1684009511']['pred'])

In [14]:
for unpred_result in collection.find({ 'label' : {'$exists' : False}}):
    unpred_result['pred'] = float(df[df['jobId'] == unpred_result['jobId']]['pred'])
    unpred_result['pred_date'] = datetime.datetime.utcnow()
    collection.save(unpred_result)

In [ ]:
pd.DataFrame(list(collection.find({ 'label' : {'$exists' : False}})))